<a href="https://colab.research.google.com/github/USCbiostats/PM520/blob/main/Lab_11_Variational_Inference_PtIV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Lucky, or: Non-Conjugate Variational Inference Pt II

Recall, that  CAVI derivations assume that our surrogate models are the result of conditional conjugacy between the expected log likelihood and the prior.

Last week, we _assumed_ that $Q_j(\theta_j)$ is in the _same_ exponential family as its corresponding prior $\Pr(\theta_j)$. We then derived the analytic expectations required for the ELBO and performed gradient ascent.

> _What if there are no closed form/analytic solutions for the expectations (i.e. ELBO)_?

One solution to this problem, is to leverage _stochastic_ gradient descent by performing Monte Carlo sampling of the necessary gradients.

## Why is this a problem? Lemme differentiate under the integral!
$$\begin{align*}
\text{ELBO}(\theta) &:= \mathbb{E}_Q\left[\log \Pr(\mathbf{X} | \mathbf{z})\right] - \mathsf{D}_{KL}(Q(\mathbf{z})||P(\mathbf{z}))\\
\nabla_{\theta}\text{ELBO}(\theta) &=
  \nabla_{\theta}\mathbb{E}_Q\left[\log \Pr(\mathbf{X} | \mathbf{z})\right]
  -\underbrace{\nabla_{\theta}\mathsf{D}_{KL}(Q(\mathbf{z})||P(\mathbf{z}))}_{\text{typically analytically tractable!}}\\
\nabla_{\theta}\mathbb{E}_Q\left[\log \Pr(\mathbf{X} | \mathbf{z})\right] &= \nabla_{\theta}\int Q_{\theta}(\mathbf{z})\log \Pr(\mathbf{X} | \mathbf{z}) d\mathbf{z}\\
  &= \int \nabla_{\theta}Q_{\theta}(\mathbf{z})\log \Pr(\mathbf{X}|\mathbf{z})d \mathbf{z} \\
  &\neq \mathbb{E}_{Q}\left[\nabla_{\theta} \log \Pr(\mathbf{X} | \mathbf{z})\right]
\end{align*}$$


Our expectation depends on the parameters $\theta$, which complicates our expression, and typically doesn't result in a known closed form solution. Additionally, by the straightforward derivation above, we see that we can't sample gradients directly as it isn't the same expression!

> _What gives?_

## Reparameterization Trick
If we can define $\mathbf{z}$ as a _deterministic_ function $g_{\theta}(ɛ) \mapsto \mathbf{z}$, then we may be able to circumvent this issue. For _location-scale_ families $f$, this is trivial! Namely, $\mathbf{z} = \mu + \sigma \circ ɛ$, where $ɛ_j \sim f(0, 1)$, $\theta = \{\mu, \sigma\}$. Now we have gone from $\mathbb{E}_{Q}\left[ \log \Pr(\mathbf{X} | \mathbf{z}) \right]$ to $\mathbb{E}_{ɛ \sim f(0, 1)}\left[ \log \Pr(\mathbf{X} | g_{\theta}(ɛ))\right]$.

> _How does this help us?_

We can use Monte-Carlo estimates of the gradient under this reparameterization to approximate the exact gradient. This can be
shown by,
$$\begin{align*}
\nabla_{\theta}\mathbb{E}_Q\left[\log \Pr(\mathbf{X} | \mathbf{z})\right] &=
  \nabla_{\theta}\mathbb{E}_{ɛ \sim f(0, 1)}\left[ \log \Pr(\mathbf{X} | g_{\theta}(ɛ))\right] \\
&= \nabla_{\theta} \int f(ɛ) \log \Pr(\mathbf{X} | g_{\theta}(ɛ)) dɛ\\
&= \int f(ɛ) \nabla_{\theta}\log \Pr(\mathbf{X} | g_{\theta}(ɛ)) dɛ\\
&= \mathbb{E}_{ɛ \sim f(0, 1)}\left[\nabla_{\theta}\log \Pr(\mathbf{X} | g_{\theta}(ɛ)) \right] \\
&\approx \dfrac{1}{T} \sum_{t=1}^T \nabla_{\theta}\log \Pr(\mathbf{X} | g_{\theta}(ɛ^t)),
\end{align*}$$
where $ɛ^t \sim f(0, 1)$.


## Lab: Logistic Regression with Normal priors on effects
We would like to perform variational inference under the following model:
$$\begin{align*}
\mathbf{y}_i | \beta &\sim \text{Bernoulli}(\text{sigmoid}(\mathbf{x}_i^T \beta))\\
\beta_j &\sim N(0, \sigma^2_b).
\end{align*}$$


Let's assume $Q(\beta) = \prod_{j=1}^p Q_j(\beta_j) = N(\beta_j | \mu_j, \sigma^2_j)$. This model doesn't exhibit (any known) closed-form expectations, so we must rely on stochastic optimization. Let's code the reparameterization trick to optimize the ELBO for this model.

In [ ]:
from typing import NamedTuple

import jax
import jax.random as rdm
import jax.nn as nn
import jax.numpy as jnp
import jax.scipy as jsp

from jax import Array
from jax.tree_util import tree_multimap
from jax.typing import ArrayLike


class PriorParams(NamedTuple):
  mean_b: Array
  var_b: Array


class PosteriorParams(NamedTuple):
  mean_b: Array
  var_b: Array


def kl_divergence(post: PosteriorParams, prior: PriorParams) -> Array:
  """KL divergence for scalar normals
  """
  pass


def complete_log_likelihood(y: ArrayLike, X: ArrayLike, beta: Array) -> float:
  """ log-likelihood of binary outcomes y, given X, beta
  """
  pass


def sample_log_likelihood(post: PosteriorParams, key: ArrayLike, y: ArrayLike, X: ArrayLike) -> float:
  n, p = X.shape
  # reparameterization trick
  beta = post.mean_b + jnp.sqrt(post.var_b) * rdm.normal(key, shape=(p,))
  return complete_log_likelihood(y, X, beta)


def fit(y: ArrayLike, X: ArrayLike, prior_var_b: float = 1e-3, step_size = 0.1, seed = 0, max_iter=500) -> PosteriorParams:
  #initialize our random keye
  n, p = X.shape
  key = rdm.PRNGKey(seed)

  # split to initalize our variational parameters
  key, init_mean_key = rdm.split(key, 2)
  post = PosteriorParams(
      mean_b = jnp.sqrt(prior_var_b) * rdm.normal(init_mean_key, (p,))
      var_b = prior_var_b * jnp.ones((p,))
  )
  prior = PriorParams(
      mean_b = jnp.zeros((p,))
      var_b = prior_var_b * jnp.ones((p,))
  )
  for epoch in range(max_iter):
    key, skey = rdm.split(key)
    # sample gradient using reparam trick for expected log like
    eloggrad = jax.grad(sample_log_likelihood)(post, skey, y, X)

    # compute exact gradient over kl div
    klgrad = jax.grad(kl_divergence)(post, prior)
    # combine gradients and take gradient step to update posterior/variational params
    grad = tree_multimap(lambda x, y: x + y, eloggrad, klgrad)
    post = tree_multimap(lambda _post, _grad: _post + step_size * _grad, post, grad)

    # sample to -compute/evaluate- the ELBO

  return post

In [ ]:
# simulate binary outcome
N, P = 250, 25
prior_var_b = 1e-2

seed = 0
key = rdm.PRNGKey(seed)
key, beta_key, x_key, y_key

beta = jnp.sqrt(prior_var_b) * rdm.normal(size=(P,))
X = rdm.normal(x_key, (N, P))

pred = X @ beta
prob = nn.sigmoid(pred)
y = rdm.bernoulli(y_key, prob)

params = fit(y, X)